In [1]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [2]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [5]:
import copy
import numpy as np
seed_torch(seed)
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 50, 5, 10, 3, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss()
init_state = copy.deepcopy(model.state_dict())



In [6]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm

import xlrd
import pandas as pd
book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
data = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

scaler = StandardScaler()
standardised_data = scaler.fit_transform(data)


x_train = standardised_data[:, 0:5]
y_train = standardised_data[:, 5:8]

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [17]:
i = 5
test_batchsizes = [1,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,1000]
train_epochs = 20
learning_rate = 0.0005 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
init_state_opt = copy.deepcopy(optimizer.state_dict())
for batchsize in test_batchsizes:
            N, D_in, H1, H2, D_out = batchsize, 5, 20, 12, 3
            model = Net(D_in, H1, H2, D_out)
            total_loss = 0
            init_state = copy.deepcopy(model.state_dict())
            for i, (train_idx, valid_idx) in enumerate(splits):
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batchsize,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batchsize,
                                                           shuffle=False)
                for epoch in range(train_epochs):
                    model.train()
                    avg_loss = 0.
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                model.eval()
                avg_val_loss = 0.
                for x_batch, y_batch in valid_loader:
                    y_pred = model(x_batch).detach()
                    avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                    final_loss = avg_val_loss
                total_loss += final_loss/(n_splits)
            print(batchsize)
            print('total loss={:.4f}'.format(total_loss))


1
total loss=1.0128
10
total loss=1.0832
20
total loss=1.3397
30
total loss=1.0480
40
total loss=1.0016
50
total loss=1.1016
60
total loss=1.0384
70
total loss=1.0723
80
total loss=1.0366
90
total loss=1.1458
100
total loss=1.0080
200
total loss=1.0930
300
total loss=1.0447
400
total loss=1.0345
500
total loss=1.0768
600
total loss=1.0408
700
total loss=1.1261
1000
total loss=1.0190


In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                